In [11]:
import os
import openai
from IPython.core.display import display, HTML

openai.api_key = "api_key"

In [12]:
import base64
import json
import requests


def github_read_file(username, repository_name, file_path, github_token=None, URL=None):
    headers = {}
    if github_token:
        headers['Authorization'] = f"token {github_token}"

    url = f'https://api.github.com/repos/{username}/{repository_name}/contents/{file_path}'
    if URL:
      url = URL
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    data = r.json()
    file_content = data['content']
    file_content_encoding = data.get('encoding')
    if file_content_encoding == 'base64':
        file_content = base64.b64decode(file_content).decode()

    return file_content

In [23]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    """This function calls ChatGPT API with a given prompt
    and returns the response back."""
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0)
    return response.choices[0].message["content"]

print ("CODE REVIEWER BOT".center(126, '-'))

# reading the default guidelines
file = open("prompt.txt",errors="ignore")
prompt = file.read()

#asking for additional guidelines or code conventions 
option = input("Do you have any other guidelines or code conventions to add? (y/n): ")
if option == "y":
    guidelines = input("Enter the guidelines or code conventions: ")
    # adding guidelines to prompt
    prompt += "11." + guidelines + "\n"

prompt += """Instructions: Based on the given guidelines, 
review the code below while generating a response take care of these points:
1. Write the output in HTML.
2. Showing all the guidelines suggest improvements in the code in a tabular format.
3. Make a section showing implementation of all the improvements in one code snippet in the next section.
4. Provide description for each improvment with examples if the code is violating a rule.
5. The columns must be Sr. No., Guidelines and modification required.
6. The color of the column heading should be HEX #A569BD color and other rows must be  HEX #F5EEF8 color.
7. The headings must be bold, in uppercase and font style Serif and other columns font style monospace.
8. Borders of the table must be black.
9. If according to guidelines modification is required then it should be written in red color.
10. If according to guidelines no modification is required then it should be written in green color.
11. If modification or may not be required then write "Not sure" and in HEX #F1C40F color.

The code is to be reviewed is:-"""

# taking code as input from user
username = input("Enter username of GitHub account: ")
repository_name = input("Enter repository_name: ")
file_path = input("Enter file_path: ")
code = github_read_file(username,repository_name,file_path)
prompt += code

response = get_completion(prompt)
display(HTML(response))

------------------------------------------------------CODE REVIEWER BOT-------------------------------------------------------
Do you have any other guidelines or code conventions to add? (y/n): n
Enter username of GitHub account: JannatKhan1
Enter repository_name: algorithms
Enter file_path: BinaryPalindromeChecker.java


Sr. No.,Guidelines,Modification Required
1,Follow Java code conventions,No
2,Replace imperative code with lambdas and streams,Yes
3,Beware of the NullPointerException,Yes
4,Directly assigning references from client code to a field,Yes
5,Handle exceptions with care,No
6,Ponder over the choice of data structures,No
7,Think twice before you expose,No
8,Code to interfaces,No
9,Don't force fit interfaces,No
10,Override hashCode when overriding equals,Yes
